In [75]:
# -*- coding: utf-8 -*-
"""
OR Final Project: Group A
"""
# Q1(b)
from gurobipy import*
import pandas as pd
import numpy as np

model = Model('OR Final Project')

I = 4

A = 16 # 一台調度車容量上限
S_H = 0.7 # 停車率的安全範圍上限比率
S_L = 0.3 # 停車率的安全範圍下限比率
L = 60 # 調度車在單位時間內移動的距離（或時間）上限
lambda_value = 500 #有改過 比較合理
M=100000
e=0.01
# 站點 i 的停車數量
#Parked = [0,200*S_L-1,100*S_L,150*S_L,20*S_L,0]
Parked = [0,55,80,40,10,0]
# 為站點 i 的車柱總數
Capacity = [0,200,100,150,20,10000]

# 調度車輛從站點 i 到站點 j 的成本 (c_ij 要等於 c_ji)
c = [[1 if i!=I+1 and i!=0 and i!=j else 0 for i in range(I+2)] if j!=I+1 and j!=0 else [0 for k in range(I+2)]for j in range(I+2)]
# print("Cost:",*c, sep='\n')

# In[] Assign variables
a=[] #調度車從i站拿走的量
b=[] #調度車放入i站的量
p=[] #進i站時調度車上的腳踏車量
q=[] #出i站時調度車上的腳踏車量
x=[] #=1代表有經過i站
y=[] #yij=1代表從i走到j
w_L=[] #各點的懲罰比率
w_H=[] #各點的懲罰比率
u=[]
v=[]

a =   model.addVars(I+2,      lb = 0, vtype = GRB.INTEGER, name = 'a')
b =   model.addVars(I+2,      lb = 0, vtype = GRB.INTEGER, name = 'b')
p =   model.addVars(I+2,      lb = 0, vtype = GRB.INTEGER, name = 'p')
q =   model.addVars(I+2,      lb = 0, vtype = GRB.INTEGER, name = 'q')

x =   model.addVars(I+2,      lb = 0, vtype = GRB.BINARY,  name = 'x')
y =   model.addVars(I+2, I+2, lb = 0, vtype = GRB.BINARY,  name = 'y') 
w_L = model.addVars(I+2,      lb = 0, vtype = GRB.CONTINUOUS, name = 'w_L')
w_H = model.addVars(I+2,      lb = 0, vtype = GRB.CONTINUOUS, name = 'w_H')

u =   model.addVars(I+2,      lb = 0, vtype = GRB.INTEGER, name = 'u')
v =   model.addVars(I+2, I+2, lb = 0, vtype = GRB.BINARY, name = 'v')#決定每個u的關聯

#虛擬點為0是起點,n+1為終點
#真正的點為1~n
model.update()
model.setObjective( quicksum(y[i,j]*c[i][j] for i in range(I+2) for j in range(I+2)) + lambda_value*quicksum(w_L[i]+w_H[i] for i in range(1,I+1)) , GRB.MINIMIZE)
     
"""路線限制!!!"""

# 兩個點都走到才有機會連在一起->有y必有x
model.addConstrs((x[i] + x[j]  >= 2*(y[i, j]+y[j, i]) for i in range(I+2) for j in range(I+2)), "C1")
# 對於起終點以外，進的點=出的點(也就是都只有一個點)
model.addConstrs(   (quicksum(y[i, k] for i in range(0,I+1))==quicksum(y[k, j] for j in range(1,I+2)) for k in range(1,I+1)   ),"C2")
# 不能到自己
model.addConstrs(   (y[i, i]==0 for i in range(I+2))   ,"C3")#這邊的寫法跟pdf上不一樣有小改ij to ii
# 不能來回走
model.addConstrs(   (y[i, j]+y[j, i] <= 1 for i in range(I+2) for j in range(I+2))   ,"C4")
# 不能跑超過一小時能跑的距離(L)
model.addConstr(   (quicksum(y[i, j]*c[i][j] for i in range(I+2) for j in range(I+2)) <= L)   ,"C5")
# 一站只能進或出一次
model.addConstrs(   (quicksum(y[i, j] for i in range(I+2))<=1 for j in range(I+2))   ,"C6-1")
model.addConstrs(   (quicksum(y[i, j] for j in range(I+2))<=1 for i in range(I+2))   ,"C6-2")
# x不為0時y要有東西
model.addConstrs(   ((quicksum(y[i, j] for j in range(I+2))+quicksum(y[j, i] for j in range(I+2))) >= x[i] for i in range(I+2))   ,"C7")#這邊的寫法跟pdf上不一樣有小改範圍

#若只跑045, x0 x4 x5等於1 y04 y45 等於1
#目標：023145-> 02315
""" 處理subtour """
model.addConstr(  (u[0] == 1), "")
#model.addConstrs( (u[I+1] >= u[i] for i in range(I+1)),"最後一點最大")
model.addConstrs( (u[i] <= (I+2) for i in range(1,I+2)) , "")
model.addConstrs( (u[i] >= 2 for i in range(1,I+2)) , "")
model.addConstrs( (u[i] - u[j] + 1 <= (M)*(1-y[i, j]) for i in range(I+2) for j in range(I+2) if i != j) , "")
# v_ij=1 if ui > uj else 0
model.addConstrs( (u[i]-u[j]<=M*v[i, j]-e for i in range(I+2) for j in range(I+2) if i != j),"每個u不一樣")
model.addConstrs( (u[i]-u[j]>=e-M*(1-v[i, j]) for i in range(I+2) for j in range(I+2) if i != j),"每個u不一樣")

"""站點bike限制!!!"""
# 只能拿或放，不能同時
model.addConstrs(   (a[i]*b[i] == 0 for i in range(I+2)) , "C8")

# 有經過才拿或放
model.addConstrs( (M*x[i] >= a[i] for i in range(I+2)), "C9-1")
model.addConstrs( (M*x[i] >= b[i] for i in range(I+2)), "C9-2")

# 懲罰
model.addConstrs((w_L[i] >= S_L - (Parked[i] - a[i] * x[i] + b[i] * x[i])/Capacity[i]    for i in range(1,I+1)), "C10-1-1")
model.addConstrs((w_L[i] >= 0 for i in range(1,I)), "")

model.addConstrs((w_H[i] >= (Parked[i] - a[i] * x[i] + b[i] * x[i])/Capacity[i] - S_H    for i in range(1,I+1)), "C10-2-1")
model.addConstrs((w_H[i] >= 0 for i in range(1,I)), "")

# 所有從各站拿走總合等於放回 卡車最後沒東西都放回虛擬點
model.addConstr(  (quicksum(a[i]*x[i] for i in range(I+2)) == quicksum(b[i]*x[i] for i in range(I+2))), "C11")
# 合理的放回車站
model.addConstrs( (b[i] +  Parked[i] <= Capacity[i] for i in range(1,I+2)), "C12-1")
# 合理地拿出車站
model.addConstrs( (a[i] <= Parked[i]                for i in range(1,I+1)), "C12-2")


""" trucks constraints """
# 進+拿-放=出
model.addConstrs( (p[i] + a[i] - b[i] == q[i] for i in range(I+2)), "C13")
# 從此站出的腳踏車量等於到下站時的腳踏車量
model.addConstrs( (q[i]*y[i,j] == p[j]*y[i,j] for i in range(I+2) for j in range(I+2) if i != j), "C14")
# 車上總車數不能超過容量
model.addConstrs( (p[i] <= A for i in range(I+2)), "C15-1")
model.addConstrs( (q[i] <= A for i in range(I+2)), "C15-2")
# 不能放下超過貨車上進場時的腳踏車量
model.addConstrs( (b[i] <= p[i] for i in range(I+2)), "")

""" Virtual points constraints """
# 起點的a,b為0
model.addConstr(  (a[0]   == 0), "")
model.addConstr(  (b[0]   == 0), "")
# 終點的a為0,b都可
model.addConstr(  (a[I+1] == 0), "")
model.addConstr(  (b[I+1] >= 0), "終點可以放車（真實的站點最後一站沒補完）")
# 起終點必經
model.addConstr(  (x[0]   == 1), "")
model.addConstr(  (x[I+1] == 1), "")
# 起點不進，終點不出
model.addConstrs( (y[i,   0] == 0 for i in range(1, I+2))  , "不能進入虛擬起點")#範圍小改
model.addConstrs( (y[I+1, i] == 0 for i in range(I+1))  , "虛擬終點不能出去")#範圍小改
# 起終虛擬點不能互通
#model.addConstr(  (y[0, I+1] == 0)                         , "不能起點直接到終點")
#model.addConstr(  (y[I+1, 0] == 0)                         , "不能終點直接到起點")
# 在起點時沒車，終點時即便原本有車也要放到沒
model.addConstr(  (p[0]   == 0), "")
model.addConstr(  (q[0]   == 0), "")
model.addConstr(  (p[I+1] >= 0), "進入終點時可以有車（真實的站點最後一站沒補完）")
model.addConstr(  (q[I+1] == 0), "")
# 起終點不懲罰
model.addConstr(  (w_L[0]   == 0), "")
model.addConstr(  (w_L[0]   == 0), "")
model.addConstr(  (w_H[I+1] == 0), "")
model.addConstr(  (w_H[I+1] == 0), "")

<gurobi.Constr *Awaiting Model Update*>

In [76]:
model.optimize()

for var in model.getVars():
    print(var.varName, '=', var.x)
print("objective value =", model.objVal)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 277 rows, 120 columns and 784 nonzeros
Model fingerprint: 0xf0798d0b
Model has 45 quadratic constraints
Variable types: 12 continuous, 108 integer (78 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  QMatrix range    [5e-03, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 1e+05]
  QRHS range       [2e-02, 5e-01]
Presolve removed 126 rows and 44 columns
Presolve time: 0.01s
Presolved: 176 rows, 84 columns, 607 nonzeros
Variable types: 5 continuous, 79 integer (49 binary)
Found heuristic solution: objective 79.1666667

Root relaxation: objective 2.000000e+00, 50 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Be

In [21]:
model.optimize()

# In[] Export result
print('Objective value: %2f'%(model.objVal))

output = pd.DataFrame(columns = ("Var", "Value"))
no = 1
for _,v in enumerate(model.getVars()):
    output.loc[no] = [v.varname, float("{:.4f}".format(v.x))]
    no += 1
    
output = pd.DataFrame(output)
print(output)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 164 rows, 78 columns and 518 nonzeros
Model fingerprint: 0x416580fd
Model has 59 quadratic constraints
Variable types: 0 continuous, 78 integer (42 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  QMatrix range    [5e-03, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
  QRHS range       [2e-02, 5e-01]
Presolved: 15 rows, 9 columns, 46 nonzeros

Continuing optimization...


Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
Objective value: 3.000000
       Var  Value
1     a[0]    0.0
2     a[1]    0.0
3    

In [27]:
u = np.full((I+2), -1) #建立一個內容為2.2 4x2 的浮點數陣列
u

[]


array([-1, -1, -1, -1, -1, -1])